<a href="https://colab.research.google.com/github/Tom984-vn/Math-symbol-recognition/blob/main/Math_formula_regcognitive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("xainano/handwrittenmathsymbols")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/handwrittenmathsymbols


In [ ]:
!apt-get install unrar -y
!unrar x /root/.cache/kagglehub/datasets/xainano/handwrittenmathsymbols/versions/2/data.rar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /root/.cache/kagglehub/datasets/xainano/handwrittenmathsymbols/versions/2/data.rar


Would you like to replace the existing file extracted_images/!/!_100071.jpg
   544 bytes, modified on 2017-01-15 16:31
with a new one
   544 bytes, modified on 2017-01-15 16:31

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit 
User break

User break


In [ ]:
path = "/content/extracted_images"
classes = os.listdir(path)
print(classes)

['sin', '!', '(', 'M', 'C', 'phi', 'o', 'A', 'S', 'sigma', 'lambda', 'lt', '0', 'gamma', 'forall', '4', 'z', 'T', '3', 'prime', 'cos', '1', 'pm', 'G', 'k', '+', 'j', 'q', 'tan', 'ldots', 'Delta', 'R', 'div', '2', 'u', 'l', 'theta', '}', 'in', 'leq', 'H', ']', 'v', 'b', '5', 'N', 'exists', 'ascii_124', '8', '=', '[', 'd', 'infty', '7', 'pi', 'y', 'beta', 'forward_slash', '6', 'sum', 'int', 'gt', 'geq', 'e', ')', 'neq', 'i', 'log', 'p', 'sqrt', 'f', 'w', 'X', ',', 'rightarrow', 'times', '-', 'lim', '{', 'alpha', '9', 'mu']


In [ ]:
from torch.utils.data import random_split
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
])
dataset = datasets.ImageFolder(root=path, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
training, test_dataset = random_split(dataset, [train_size, test_size])

#create validation dataset 10%
val_size = int(0.1 * len(training))
train_size = len(training) - val_size
train_dataset, val_dataset = random_split(training, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

print(f"Total: {len(dataset)} | Train: {len(train_dataset)}")
print(f"Classes: {dataset.classes}")

Total: 375974 | Train: 270702
Classes: ['!', '(', ')', '+', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'A', 'C', 'Delta', 'G', 'H', 'M', 'N', 'R', 'S', 'T', 'X', '[', ']', 'alpha', 'ascii_124', 'b', 'beta', 'cos', 'd', 'div', 'e', 'exists', 'f', 'forall', 'forward_slash', 'gamma', 'geq', 'gt', 'i', 'in', 'infty', 'int', 'j', 'k', 'l', 'lambda', 'ldots', 'leq', 'lim', 'log', 'lt', 'mu', 'neq', 'o', 'p', 'phi', 'pi', 'pm', 'prime', 'q', 'rightarrow', 'sigma', 'sin', 'sqrt', 'sum', 'tan', 'theta', 'times', 'u', 'v', 'w', 'y', 'z', '{', '}']


Encoder:
  - CNN (e.g., ResNet18 or a lightweight custom CNN)
  - Outputs spatial feature map

Decoder:
  - Transformer or LSTM with attention over encoder features
  - Predicts sequence of math tokens
  

In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channel = 1, num_classes = 119):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1,out_channels= 32, kernel_size=3, stride=1, padding=1)
    self.conv2drop = nn.Dropout2d(p=0.2)
    self.drop = nn.Dropout(p=0.2)
    self.bn1 = nn.BatchNorm2d(32)
    self.bn2 = nn.BatchNorm2d(64)
    self.bn3 = nn.BatchNorm2d(128)
    self.conv2 = nn.Conv2d(32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(64,out_channels=128, kernel_size=3, stride=1, padding=1)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.ReLU = nn.ReLU()
    self.fc1 = nn.Linear(128*5*5, 256)
    self.fc2 = nn.Linear(256, 119)
  def forward(self, x):
    x = self.pool(self.ReLU(self.bn1(self.conv1(x))))
    x = self.pool(self.ReLU(self.bn2(self.conv2(x))))
    x = self.pool(self.ReLU(self.bn3(self.conv3(x))))
    x = self.conv2drop(x)
    x = x.view(x.size(0), -1)
    x = self.ReLU(self.fc1(x))
    x = self.drop(x)
    x = self.fc2(x)
    return x
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def validation(model, val_loader, criterion):
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
      for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        total_val_loss += loss.item()
    return total_val_loss / len(val_loader)

In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5

In [ ]:
for epoch in range(num_epochs):
  total_loss = 0
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    if batch_idx % 10 == 0:
      print(f"Epoch: {epoch+1}/{num_epochs}, Batch: {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")
  val_loss = validation(model, val_loader, criterion)
%clear
print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")
print(f"Val loss: {val_loss}")

Epoch: 1/5, Batch: 0/4230, Loss: 4.782042503356934
Epoch: 1/5, Batch: 10/4230, Loss: 3.243180274963379
Epoch: 1/5, Batch: 20/4230, Loss: 2.7343952655792236
Epoch: 1/5, Batch: 30/4230, Loss: 1.984082818031311
Epoch: 1/5, Batch: 40/4230, Loss: 1.3513957262039185
Epoch: 1/5, Batch: 50/4230, Loss: 1.6412774324417114
Epoch: 1/5, Batch: 60/4230, Loss: 1.404362440109253
Epoch: 1/5, Batch: 70/4230, Loss: 1.1917973756790161
Epoch: 1/5, Batch: 80/4230, Loss: 1.510817289352417
Epoch: 1/5, Batch: 90/4230, Loss: 1.4157216548919678
Epoch: 1/5, Batch: 100/4230, Loss: 0.9803276658058167
Epoch: 1/5, Batch: 110/4230, Loss: 0.7901968955993652
Epoch: 1/5, Batch: 120/4230, Loss: 1.2073813676834106
Epoch: 1/5, Batch: 130/4230, Loss: 1.047498345375061
Epoch: 1/5, Batch: 140/4230, Loss: 0.4711601138114929
Epoch: 1/5, Batch: 150/4230, Loss: 0.9895057678222656
Epoch: 1/5, Batch: 160/4230, Loss: 0.9092808961868286
Epoch: 1/5, Batch: 170/4230, Loss: 0.5669831037521362
Epoch: 1/5, Batch: 180/4230, Loss: 1.11854100

In [ ]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)
      print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
      model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Streaming output truncated to the last 5000 lines.
Got 16541 / 25984 with accuracy 63.66
Got 16579 / 26048 with accuracy 63.65
Got 16625 / 26112 with accuracy 63.67
Got 16667 / 26176 with accuracy 63.67
Got 16710 / 26240 with accuracy 63.68
Got 16751 / 26304 with accuracy 63.68
Got 16795 / 26368 with accuracy 63.69
Got 16833 / 26432 with accuracy 63.68
Got 16874 / 26496 with accuracy 63.69
Got 16913 / 26560 with accuracy 63.68
Got 16958 / 26624 with accuracy 63.69
Got 17004 / 26688 with accuracy 63.71
Got 17046 / 26752 with accuracy 63.72
Got 17087 / 26816 with accuracy 63.72
Got 17130 / 26880 with accuracy 63.73
Got 17163 / 26944 with accuracy 63.70
Got 17199 / 27008 with accuracy 63.68
Got 17236 / 27072 with accuracy 63.67
Got 17277 / 27136 with accuracy 63.67
Got 17322 / 27200 with accuracy 63.68
Got 17367 / 27264 with accuracy 63.70
Got 17410 / 27328 with accuracy 63.71
Got 17444 / 27392 with accuracy 63.68
Got 17487 / 27456 with accuracy 63.69
Got 17524 / 27520 with accuracy 63.68